In [20]:
from __future__ import division
import h5py
import struct
import ctypes
import pymorton as pm
from numpy import array

In [21]:
dir(pymorton)

['DIVISORS',
 '_',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__part1by1',
 '__part1by2',
 '__path__',
 '__unpart1by1',
 '__unpart1by2',
 'deinterleave2',
 'deinterleave3',
 'deinterleave_latlng',
 'divisor',
 'interleave',
 'interleave2',
 'interleave3',
 'interleave_latlng']

In [24]:
N = 1024
zmin = pm.interleave3(N//2-1,0,0)
zmax = pm.interleave3(N-1,N//2-1,N//2-1)

In [25]:
print(zmin,zmax)
print(pm.deinterleave3(zmin),pm.deinterleave3(zmax))

(19173961, 268435455)
((511, 0, 0), (1023, 511, 511))


In [ ]:


def getblob(x, y, z, blobsize, h5file):
    #definition used to grab a blob in xyz order.
    u = h5file["U"][x:x+blobsize,y:y+blobsize,z:z+blobsize]
    v = h5file["V"][x:x+blobsize,y:y+blobsize,z:z+blobsize]
    w = h5file["W"][x:x+blobsize,y:y+blobsize,z:z+blobsize]
    buffer = ctypes.create_string_buffer(6144) #hardcoded for speed
    index = 0
    offset = 0
    #print ("First val is: " + str(u[0,0,0]))
    for k in range(0, blobsize):
        for j in range(0, blobsize):
            for i in range (0, blobsize):
                #import pdb;pdb.set_trace()
                struct.pack_into('f'*3, buffer, offset,u[k,j,i],v[k,j,i],w[k,j,i])
                offset = offset + 12
    return buffer
#Test it out!

h5file = h5py.File("outpen/OUTPEN_04.h5") #Open for read only
blobsize = 8
#blob = getblob(0,0,0,blobsize, h5file)

f = open('iso4096-102.bin', 'wb+')
zblob = blobsize * blobsize * blobsize
#interleave(x,y,z)
zmax = pymorton.interleave3(1023,511,511)
zmin = pymorton.interleave3(512,0,0)
for zindex in range(zmin,zmax, zblob):
    print ("zindex = " + str(zindex))
    #print ("coord = " + str(pymorton.deinterleave3(zindex)))
    coord = pymorton.deinterleave3(zindex)
    blob = getblob(coord[2], coord[1], coord[0], blobsize, h5file)
    f.write(blob)
    #import pdb;pdb.set_trace()

f.close()
